# 广播，布尔掩码



前面的向量化操作可以减少缓慢的python循环，另一种构造向量化操作的方式是广播，广播与其说是一种运算，倒不如说是一种处理方式，设想这样一种场景，我试图把【1，2】加【2】，我们发现这两个向量的维度不一样，或者说这两个矩阵不等大，但是我们仍试图把他俩加起来，np仍然会给你一个结果，但是这个结果将会是【1，2】加【2，2】的结果，就如下面的代码

In [1]:
import numpy as np

x = np.array([1,2])
y = np.array([2])
print(x)
print(y)
print(x+y)
print(x+y-x)

[1 2]
[2]
[3 4]
[2 2]


np在执行加法的时候把y从【2】变成了【2，2】，我们说y就经历了一次广播，简言之，让其变大，以适合另一个矩阵的形状，再进行运算，即广播规则，下面的一个例子是对两个矩阵都进行广播，然后得到公共的形状，再进行加减，这里需要注意的是，这个运算并没有实际的在内存中发生，y不会因为被广播而改变

In [2]:
x = np.array([1,2,3])
y = np.array(x[:,np.newaxis])
print(x)
print(y)
print(x+y)

[1 2 3]
[[1]
 [2]
 [3]]
[[2 3 4]
 [3 4 5]
 [4 5 6]]


详细的说，广播规则有如下三条：
1. 若两个数组维度不等，那么小的那个会在左侧补1  :  例如，形状为 `(2, 3)` 的二维数组与形状为 `(4, 1, 3)` 的三维数组运算时，前者会被填充为 `(1, 2, 3)`。
2. 若两个数组在任何维度都不匹配，数组的形状会沿着维度为1的那个方向扩展（譬如变成2），以匹配形状  :  例如，形状为 `(2, 1)` 的数组与形状为 `(2, 3)` 的数组运算时，前者会被扩展为 `(2, 3)`。

3. 若两个数组形状在任何维度上都不匹配且没有维度等于1，那么报错  :  例如，形状为 `(2, 3)` 和 `(3, 2)` 的数组无法直接进行广播运算。

In [3]:
x = np.ones((4,1,3))
print(x)
print(x.shape)
y = np.arange(6)
y = y.reshape((2, 3))  # 正确方式：使用 reshape 改变形状
z = x + y
k = z - x
print()
print(y)
print(y.shape)
print()
print(z)
print(z.shape)


[[[1. 1. 1.]]

 [[1. 1. 1.]]

 [[1. 1. 1.]]

 [[1. 1. 1.]]]
(4, 1, 3)

[[0 1 2]
 [3 4 5]]
(2, 3)

[[[1. 2. 3.]
  [4. 5. 6.]]

 [[1. 2. 3.]
  [4. 5. 6.]]

 [[1. 2. 3.]
  [4. 5. 6.]]

 [[1. 2. 3.]
  [4. 5. 6.]]]
(4, 2, 3)


In [4]:
M = np.ones((2, 3))
a = np.arange(3)
print(M)
print(M.shape)
print()
print(a)
print(a.shape)
print()
print(M+a)

[[1. 1. 1.]
 [1. 1. 1.]]
(2, 3)

[0 1 2]
(3,)

[[1. 2. 3.]
 [1. 2. 3.]]


现在我们考虑这两个数组的运算，它们的形状如下：

- `M.shape` 为 `(2, 3)`
- `a.shape` 为 `(3,)`

根据规则 1，数组 `a` 的维度较少，因此我们在其左侧用 1 填充维度：

- `M.shape` 保持 `(2, 3)`
- `a.shape` 变为 `(1, 3)`

根据规则 2，此时第一个维度（即行维度）的大小不匹配，因此需要将此维度扩展以匹配：

- `M.shape` 保持 `(2, 3)`
- `a.shape` 变为 `(2, 3)`

此时两个数组的形状完全匹配，最终运算结果的形状将为 `(2, 3)`：

接下来，让我们看一个两个数组不兼容的示例： 
数组的形状如下：

- `M` 的形状是 `(3, 2)`
- `a` 的形状是 `(3,)`

同样，规则 1 告诉我们，必须在 `a` 的形状左侧用 1 进行填充：

- `M` 的形状保持为 `(3, 2)`
- `a` 的形状变为 `(1, 3)`

根据规则 2，`a` 的第一个维度随后会被拉伸以匹配 `M` 的该维度：

- `M` 的形状保持为 `(3, 2)`
- `a` 的形状变为 `(3, 3)`

现在我们遇到了规则 3 —— 最终的形状不匹配，所以这两个数组是不兼容的，我们可以通过尝试进行此运算来观察到这一点： 

In [5]:

b = np.ones((3, 2))
a = np.arange(3)
print(a)
print(a.shape)
print()
print(b)
print(b.shape)
print(a+b)


[0 1 2]
(3,)

[[1. 1.]
 [1. 1.]
 [1. 1.]]
(3, 2)


ValueError: operands could not be broadcast together with shapes (3,) (3,2) 

数据中心化
在上一节中中，我们了解到通用函数能让NumPy用户无需显式编写运行缓慢的Python循环。而广播机制进一步拓展了这一能力。

在数据科学领域，一个常见的例子是**从数据数组中逐行减去均值**。

假设我们有10个观测值，每个观测值包含3个特征值，我们会将这些数据存储在一个 $10 \times 3$ 的数组中： 

In [6]:
rng = np.random.default_rng(seed=1701)    #这是指定一个种子以让每次都得到随机的但是相同的结果
#随机种子的作用是为伪随机数生成器提供初始值，确保在相同种子下能重现相同的随机序列
X = rng.random((10, 3))
print(X)
print(X.shape)

Xmean = X.mean(0)    # 沿第0轴(行方向)求均值，保留3个特征，这代表这个操作是沿着行增大的方向进行，也就是竖着发生的
print(Xmean)
print(Xmean.shape)
print()
X_centered = X - Xmean   #广播发生在这里：X(10,3) - Xmean(3,)
print(X_centered.mean(0))

[[0.4020733  0.30563311 0.67668051]
 [0.15821208 0.79247763 0.09419469]
 [0.36753944 0.06388928 0.96431608]
 [0.35200998 0.54550343 0.88597945]
 [0.57016965 0.26614394 0.8170382 ]
 [0.55906652 0.06387035 0.84877751]
 [0.89414484 0.18920785 0.23660015]
 [0.16502896 0.56583856 0.29513111]
 [0.29078012 0.90079544 0.59992434]
 [0.09133896 0.00578466 0.97096222]]
(10, 3)
[0.38503638 0.36991443 0.63896043]
(3,)

[ 4.99600361e-17 -4.44089210e-17  0.00000000e+00]


## 布尔掩码

如果你试图基于某些准则来抽取，修改，计数或操作其他的数组值时，就可以利用布尔掩码，譬如一个最简单的例子是检查数据中有多少值大于一个给定值，这里的x<0.5就是计算了x的布尔掩码，其逻辑研究数组值关于0.5的大小，大于了就是成真，成真了就在相应的位置里给出Ture，反之就是False，紧接着我们把这个布尔掩码赋值给了y，于是就有

In [15]:
rng = np.random.default_rng(seed=1701)   
X = rng.random((10, 3))
print(X)
print(X.shape)
print()
y = X < 0.5
print(y)

[[0.4020733  0.30563311 0.67668051]
 [0.15821208 0.79247763 0.09419469]
 [0.36753944 0.06388928 0.96431608]
 [0.35200998 0.54550343 0.88597945]
 [0.57016965 0.26614394 0.8170382 ]
 [0.55906652 0.06387035 0.84877751]
 [0.89414484 0.18920785 0.23660015]
 [0.16502896 0.56583856 0.29513111]
 [0.29078012 0.90079544 0.59992434]
 [0.09133896 0.00578466 0.97096222]]
(10, 3)

[[ True  True False]
 [ True False  True]
 [ True  True False]
 [ True False False]
 [False  True False]
 [False  True False]
 [False  True  True]
 [ True False  True]
 [ True False False]
 [ True  True False]]


类似的，你不难想到<= , >= , == , < , > , != 也各自是布尔掩码，上面的代码中我们把布尔掩码数组赋值到了y中去，但是你其实也可以不这样做，直接使用x<0.5，np也会给你返回x对应的布尔数组

In [16]:
rng = np.random.default_rng(seed=1701)   
X = rng.random((10, 3))
print(X)
print()
print(X < 0.5)

[[0.4020733  0.30563311 0.67668051]
 [0.15821208 0.79247763 0.09419469]
 [0.36753944 0.06388928 0.96431608]
 [0.35200998 0.54550343 0.88597945]
 [0.57016965 0.26614394 0.8170382 ]
 [0.55906652 0.06387035 0.84877751]
 [0.89414484 0.18920785 0.23660015]
 [0.16502896 0.56583856 0.29513111]
 [0.29078012 0.90079544 0.59992434]
 [0.09133896 0.00578466 0.97096222]]

[[ True  True False]
 [ True False  True]
 [ True  True False]
 [ True False False]
 [False  True False]
 [False  True False]
 [False  True  True]
 [ True False  True]
 [ True False False]
 [ True  True False]]


此后我们就可以根据这个列表的布尔掩码来对列表进行相关的操作，譬如我们试图统计布尔数组中True的个数，这会对应到原数组中满足条件的元素的个数，我们可以用count-nonzero函数

In [26]:
rng = np.random.default_rng(seed=1701)   
X = rng.random((4, 4))
print(X)
print()
y = X < 0.5
print(y)
print()
print(np.count_nonzero(y))

[[0.4020733  0.30563311 0.67668051 0.15821208]
 [0.79247763 0.09419469 0.36753944 0.06388928]
 [0.96431608 0.35200998 0.54550343 0.88597945]
 [0.57016965 0.26614394 0.8170382  0.55906652]]

[[ True  True False  True]
 [False  True  True  True]
 [False  True False False]
 [False  True False False]]

8


另一个途径是使用sum函数，一个true会被记录为1，反之false则会记录为0，这里需要注意的是，这个函数和下面的几个都要用np的版本而非python自带的

In [27]:
rng = np.random.default_rng(seed=1701)   
X = rng.random((4, 4))
print(X)
print()
print(np.sum(X < 0.5))

[[0.4020733  0.30563311 0.67668051 0.15821208]
 [0.79247763 0.09419469 0.36753944 0.06388928]
 [0.96431608 0.35200998 0.54550343 0.88597945]
 [0.57016965 0.26614394 0.8170382  0.55906652]]

8


这个途径的好处是，sum支持输入参数，允许我们控制操作发生的方向，这一点的操作和前面是一样的

In [39]:
rng = np.random.default_rng(seed=1701)   
X = rng.random((4, 3))
print(X)
y = X > 0.5
print()
print(np.sum(y,axis=0))
print()
print(np.sum(y,axis=1))

[[0.4020733  0.30563311 0.67668051]
 [0.15821208 0.79247763 0.09419469]
 [0.36753944 0.06388928 0.96431608]
 [0.35200998 0.54550343 0.88597945]]

[0 2 3]

[1 1 1 2]


上面的两个数组代表着：沿着平行于每行增加的方向（也就是竖着从上到下，逐列的数），有多少个数是大于0.5的，和：沿着列增加的方向（也就是横着从左到右，逐行的数）有多少的数大于0.5

相较于“任意”和“存在”，np中也给出了自己的版本，all代表“全部满足则为真”，any代表“任有一个满足则为真”，这两个在此处的例子就是快速检查数据符不符合要求

In [40]:
rng = np.random.default_rng(seed=1701)   
X = rng.random((4, 3))
print(X)
y = X > 0.9
print()
print(np.any(y))
print()
print(np.all(y))

[[0.4020733  0.30563311 0.67668051]
 [0.15821208 0.79247763 0.09419469]
 [0.36753944 0.06388928 0.96431608]
 [0.35200998 0.54550343 0.88597945]]

True

False


特别的，这俩函数也可以给他指定操作发生的行或是列，即下面这个例子

In [43]:
rng = np.random.default_rng(seed=1701)   
X = rng.random((4, 3))
print(X)
y = X > 0.5
print()
print(np.any(y,axis=0))   #存在一个元素大于0.5，在每个行数增加的方向上
print()
print(np.any(y,axis=1))  #存在一个元素大于0.5，在每个列数增加的方向上
print()
print(np.all(y,axis=0))  #任意的元素均大于0.5，在每个行数增加的方向上
print()
print(np.all(y,axis=1))  #任意的元素均大于0.5，在每个列数增加的方向上

[[0.4020733  0.30563311 0.67668051]
 [0.15821208 0.79247763 0.09419469]
 [0.36753944 0.06388928 0.96431608]
 [0.35200998 0.54550343 0.88597945]]

[False  True  True]

[ True  True  True  True]

[False False False]

[False False False False]


你或许会想到如果我希望得到一个区间内的结果，这时我应该先确定不大于区间上界的，再确定不小于区间下界的，然后将他俩交一起，这里我会告诉你，python的原生布尔运算符可以解决交并补的这些问题，和之前我们说过的算术运算符一样，np重载了这些玩意，你写出来python会自动认为是np的版本，不需要像sum等那样额外注上

In [48]:
rng = np.random.default_rng(seed=1701)   
X = rng.random((4, 3))
print(X)
y = (X > 0.1)&(X < 0.6)    #大于0.1且小于0.6的元素
print()
print(y)
y = (X < 0.1)|(X > 0.6)    #小于0.1或大于0.6的元素
print()
print(y)
print()
print(~y)   #对刚才的y取非

[[0.4020733  0.30563311 0.67668051]
 [0.15821208 0.79247763 0.09419469]
 [0.36753944 0.06388928 0.96431608]
 [0.35200998 0.54550343 0.88597945]]

[[ True  True False]
 [ True False False]
 [ True False False]
 [ True  True False]]

[[False False  True]
 [False  True  True]
 [False  True  True]
 [False False  True]]

[[ True  True False]
 [ True False False]
 [ True False False]
 [ True  True False]]


In [49]:
rng = np.random.default_rng(seed=1701)   
X = rng.random((4, 3))
print(X)
condition1 = (X > 0.3)  # 大于0.3的元素
condition2 = (X < 0.5)  # 小于0.5的元素
xor_result = condition1 ^ condition2  # 异或：满足其中一个条件但不满足另一个
print("\nX > 0.3:")
print(condition1)
print("\nX < 0.5:")
print(condition2)
print("\nX > 0.3 ^ X < 0.5:")
print(xor_result)

[[0.4020733  0.30563311 0.67668051]
 [0.15821208 0.79247763 0.09419469]
 [0.36753944 0.06388928 0.96431608]
 [0.35200998 0.54550343 0.88597945]]

X > 0.3:
[[ True  True  True]
 [False  True False]
 [ True False  True]
 [ True  True  True]]

X < 0.5:
[[ True  True False]
 [ True False  True]
 [ True  True False]
 [ True False False]]

X > 0.3 ^ X < 0.5:
[[False False  True]
 [ True  True  True]
 [False  True  True]
 [False  True  True]]


当然你可能会好奇，我为什么不用and和or呢？这俩可比&和|在python原生语法中有用的多，这里需要指出的是，and和or两个关键字是用来比对两个对象相同或不同的，&和|则会逐位比对这两个对象的比特位的异同，这俩的不同举一个最简单的例子是，and和or会把任何非零整数都当作True，因此你用and和or比较42和0，结果将会是True或是False，但是&和|则会给你一个二进制串，其每个位是42和0的二进制表示逐位进行&和|运算的结果

In [60]:
print(bool(42 and 0))
print(bool(42 or 0))
print(bin(42),bin(59))
print(bin(42 & 59))
print(bin(42 | 59))
print(bin(42 ^ 59))
print(bin(~42))

False
True
0b101010 0b111011
0b101010
0b111011
0b10001
-0b101011


布尔掩码的最后一个用途是作为切分的索引，我们知道x【】会得到他的一个切片，而在其中输入他对应条件的布尔掩码，就能得到原数组中满足条件的元素的集合

In [62]:
rng = np.random.default_rng(seed=1701)   
X = rng.random((4, 3))
print(X)
print()
print(X < 0.5)
print()
print(X[X < 0.5])

[[0.4020733  0.30563311 0.67668051]
 [0.15821208 0.79247763 0.09419469]
 [0.36753944 0.06388928 0.96431608]
 [0.35200998 0.54550343 0.88597945]]

[[ True  True False]
 [ True False  True]
 [ True  True False]
 [ True False False]]

[0.4020733  0.30563311 0.15821208 0.09419469 0.36753944 0.06388928
 0.35200998]


### 数组排序

Python有几个用于对列表和其他可迭代对象进行排序的内置函数和方法。sorted函数接受一个列表并返回其排序后的版本，列表的sort方法会对列表进行原地排序，相对的，np里也有自己的版本，np.sort 函数类似于 Python 内置的 sorted 函数，它能够高效地返回一个数组的已排序副本：

In [64]:
x = np.array([2, 1, 4, 3, 5])
np.sort(x)

array([1, 2, 3, 4, 5])

与 Python 列表的 sort 方法类似，你也可以使用数组的 sort 方法对数组进行原地排序，但这个方法不返回任何东西

In [69]:
x = np.array([2, 1, 4, 3, 5])
x.sort()
print(x)
print(x.sort())

[1 2 3 4 5]
None


一个相关的函数是 argsort，它返回的是排序后元素的索引，人话说就是这个位置的元素，在未排序的数组中本来的位置是哪里，但其并没有在原数组的位置上发生排序，你还是得再排序一次

In [72]:
x = np.array([2, 1, 4, 3, 5])
i = np.argsort(x)
print(x)
print(i)
print(np.sort(x))

[2 1 4 3 5]
[1 0 3 2 4]
[1 2 3 4 5]


还有一种方法是，你刚才得到的索引配合切片得到排序后的数组

In [74]:
x = np.array([2, 1, 4, 3, 5])
i = np.argsort(x)
print(x)
print(i)
print(x[i])

[2 1 4 3 5]
[1 0 3 2 4]
[1 2 3 4 5]


如果是倒序呢？

In [75]:
x[i[::-1]]

array([5, 4, 3, 2, 1])

np.sort 函数还有一个用法是能够使用 axis 参数沿着多维数组的特定行或列进行排序

In [78]:
rng = np.random.default_rng(seed=42)
X = rng.integers(0, 10, (4, 4))
print(X)
print()
print(np.sort(X, axis=0))
print()
print(np.sort(X, axis=1))

[[0 7 6 4]
 [4 8 0 6]
 [2 0 5 9]
 [7 7 7 7]]

[[0 0 0 4]
 [2 7 5 6]
 [4 7 6 7]
 [7 8 7 9]]

[[0 4 6 7]
 [0 4 6 8]
 [0 2 5 9]
 [7 7 7 7]]


部分排序：分隔,
有时候，我们并不想对整个数组进行排序，而只是想找出数组中 k 个最小的值。NumPy 的 np.partition 函数可以实现这一需求。np.partition 函数接受一个数组和一个数字 K 作为参数；其返回的新数组中，左侧是 K 个最小的值，右侧是其余的值：¶

In [79]:
x = np.array([7, 2, 3, 1, 6, 5, 4])
np.partition(x, 3)

array([1, 2, 3, 4, 5, 6, 7])

请注意，结果数组中的前三个值是原数组中最小的三个值，数组的其余位置包含剩余的值。在这两个分区内，元素的顺序是任意的。

与排序类似，我们可以沿着多维数组的任意轴进行分区操作：¶

In [86]:
rng = np.random.default_rng(seed=42)
X = rng.integers(0, 10, (3, 7))
print(X)
print()
print(np.partition(X, 2, axis=1))

[[0 7 6 4 4 8 0]
 [6 2 0 5 9 7 7]
 [7 7 5 1 8 4 5]]

[[0 0 4 4 6 7 8]
 [0 2 5 6 7 7 9]
 [1 4 5 5 7 7 8]]


### 花式索引

前面的章节讨论了如何使用简单索引（例如 arr[0]）、切片（例如 arr[:5]）以及布尔掩码（例如 arr[arr > 0]）来访问和修改数组的部分元素。现在，我们传入索引数组来替代单个标量。这使我们能够非常快速地访问和修改数组值中复杂的子集。

假设我想要访问一个已知数组中三个不同的元素，显然我们可以直接按照他的位置找到他，并包装成一个新数组

In [88]:
rng = np.random.default_rng(seed=1701)
x = rng.integers(100, size=10)
print(x)
y = np.array([x[3], x[7], x[2]])
print()
print(y)

[90 40  9 30 80 67 39 15 33 79]

[30 15  9]


或者，我们可以传入一个包含索引的单个列表或数组，以获得相同的结果

In [90]:
rng = np.random.default_rng(seed=1701)
x = rng.integers(100, size=10)
print(x)
ind = [3, 7, 2]
y = x[ind]
print()
print(y)


[90 40  9 30 80 67 39 15 33 79]

[30 15  9]


这种方式的好处是，你可以指定结果的形状，结果将变成你索引的形状

In [92]:
rng = np.random.default_rng(seed=1701)
x = rng.integers(100, size=10)
print(x)
ind = np.array([[3, 7],
                [2, 5]])
y = x[ind]
print()
print(y)

[90 40  9 30 80 67 39 15 33 79]

[[30 15]
 [ 9 67]]


这一点也适用于二维数组

In [98]:
# 创建一个简单的3x3二维数组
arr = np.array([[1, 2, 3],
                [4, 5, 6],
                [7, 8, 9]])

# 花式索引：选择第0行和第2行
rows = np.array([0, 2])
print(arr[rows])

# 花式索引：选择第1列和第2列
cols = np.array([1, 2])
print("\n", arr[:, cols])

[[1 2 3]
 [7 8 9]]

 [[2 3]
 [5 6]
 [8 9]]


我们前面提到的广播规则在这里也会生效，当行索引和列索引形状不同时，其会按照广播的规则得到一个形状匹配的结果

In [102]:
# 创建3x3数组
arr = np.array([[1, 2, 3],
                [4, 5, 6],
                [7, 8, 9]])

# 广播示例1：行索引和列索引形状不同时会广播
rows = np.array([[0], [2]])  # 形状(2,1)
print("\n", arr[rows])
cols = np.array([1, 2])      # 形状(2,)
print("\n", arr[cols ])
print("\n", arr[rows, cols])  # 行取rows指定的行，列取col指定的列，
#但是形状将会是两个索引的广播结果


 [[[1 2 3]]

 [[7 8 9]]]

 [[4 5 6]
 [7 8 9]]

 [[2 3]
 [8 9]]


rows初始形状为(2,1)，cols初始形状为(2,)（即(2,)等同于(1,2)），

第1维度（最右）：rows的1与cols的2 → 将rows的该维度复制扩展为2，

第0维度：rows的2与cols的1 → 将cols的该维度复制扩展为2，

就等价于：

[[ [0,0], [2,2] ],  # 扩展后的行坐标

   [ [1,2], [1,2] ]] # 扩展后的列坐标

### 组合索引


为了实现更强大的操作，花式索引可以与我们之前学过的其他索引方式结合使用。例如，给定数组 X，我们可以将花式索引和简单索引结合起来使用，还可以将花式索引与切片操作结合起来

In [108]:
X = np.arange(12).reshape((3, 4))
print(X)
print()
print(X[2, [2, 0, 1]])  #行取第二行，列取索引
print()
print(X[1:, [2, 0, 1]])   #行切满，但每行都只取索引值

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]

[10  8  9]

[[ 6  4  5]
 [10  8  9]]


以及掩码

In [112]:
arr = np.arange(16).reshape(4,4)
print("原始数组:\n", arr)

# 创建行掩码（选择第1行和第3行）
row_mask = np.array([False, True, False, True]) 

# 创建列索引（选择第0列和第2列）
col_indices = np.array([0, 2])  

# 组合使用：先通过掩码筛选行，再通过花式索引选择列
result = arr[row_mask][:, col_indices]
print("\n掩码+花式索引结果:\n", result)

原始数组:
 [[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]

掩码+花式索引结果:
 [[ 4  6]
 [12 14]]


利用花式索引修改值：
正如花式索引可用于访问数组的某些部分一样，它也能够用来修改数组的某些部分。

In [113]:
x = np.arange(10)
i = np.array([2, 1, 8, 4])
x[i] = 99
print(x)

[ 0 99 99  3 99  5  6  7 99  9]


类似的

In [115]:
# 生成5x5随机矩阵（1-10范围）
matrix = np.random.randint(1, 11, size=(5,5))
print("原始矩阵:\n", matrix)

# 创建布尔掩码（标记不大于5的元素）
mask = matrix <= 5

# 通过掩码修改值
matrix[mask] = -1
print("\n修改后矩阵:\n", matrix)

原始矩阵:
 [[10  3  7  7  3]
 [ 9  3  2  7  3]
 [ 7  2  5  2  5]
 [ 2  7  2 10  9]
 [ 3  6  7  4  8]]

修改后矩阵:
 [[10 -1  7  7 -1]
 [ 9 -1 -1  7 -1]
 [ 7 -1 -1 -1 -1]
 [-1  7 -1 10  9]
 [-1  6  7 -1  8]]


不过要注意的是，在这些操作中重复出现的索引可能会导致一些出乎意料的结果。看看下面这个例子：

In [116]:
x = np.zeros(10)
x[[0, 0]] = [4, 6]
print(x)

[6. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


数字 4 哪去了呢？这个操作首先将 x[0] 赋值为 4，接着又将 x[0] 赋值为 6。

再看看下面这个操作：

In [119]:
x = np.zeros(10)
x[[0, 0]] = [4, 6]
print(x)
i = [2, 3, 3, 4, 4, 4]
x[i] += 1
print(x)

[6. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[6. 0. 1. 1. 1. 0. 0. 0. 0. 0.]


你可能会认为 x[3] 的值应该是 2，x[4] 的值应该是 3，这是一个关于NumPy花式索引(fancy indexing)的重要特性问题。出现这种现象的原因是：当使用 x[i] += 1 时，NumPy实际上会先创建 x[i] 的临时副本，对副本中的每个对应位置加1（包括重复索引），最后将结果一次性写回原数组，因此每个索引位置最终只被更新一次

你可以使用通用函数（ufuncs）的 at 方法，这是一个专门用来解决重复索引的累加操作的方法

In [120]:
x = np.zeros(10)
i = [2, 3, 3, 4, 4, 4]
np.add.at(x, i, 1)
print(x)

[0. 0. 1. 2. 3. 0. 0. 0. 0. 0.]
